In [ ]:
import requests
import os
import random
import pandas as pd
import time
from bs4 import BeautifulSoup

# Load API keys from environment variables (recommended for security)
CLOUDFLARE_API_KEY = os.getenv("CLOUDFLARE_API_KEY", "CrlD3BevWJZpQDRlv_2FI6IgsmxHeQWFaCLU9Y5y")
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY", "34cee743aef7ee0f1d8526767a09593d")
SPOONACULAR_API_KEY = os.getenv("SPOONACULAR_API_KEY", "99888bc7ca0a4bcbbfa069886fcd371d")

# Cloudflare API URLs
CLOUDFLARE_VERIFY_URL = "https://api.cloudflare.com/client/v4/user/tokens/verify"
CLOUDFLARE_AI_URL = "https://api.cloudflare.com/client/v4/accounts/32c39b1f7ff91c5ebec63fedf7342109/ai/run/@cf/meta/llama-3-8b-instruct"

# User input for location
CITY = "San Francisco"

def get_user_city():
    city = input("Enter your city: ").strip()
    if not city:
        print("❌ Invalid city name. Using default: San Francisco.")
        return "San Francisco"
    return city

# Verify Cloudflare API Token
def verify_cloudflare_token():
    headers = {
        "Authorization": f"Bearer {CLOUDFLARE_API_KEY}",
        "Content-Type": "application/json"
    }
    response = requests.get(CLOUDFLARE_VERIFY_URL, headers=headers)
    if response.status_code == 200:
        print("✅ Cloudflare API Token Verified!")
        return True
    else:
        print(f"❌ Cloudflare API Token Verification Failed: {response.status_code}")
        return False

# Fetch weather data
def fetch_weather_data(city):
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={OPENWEATHER_API_KEY}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Error fetching weather data: {response.status_code}")
        return None

# Get food type based on weather condition
def get_food_type(weather_desc):
    weather_desc = weather_desc.lower()
    if "rain" in weather_desc:
        return "soup"
    elif "clear" in weather_desc or "sun" in weather_desc:
        return "salad"
    elif "cloud" in weather_desc:
        return "stew"
    elif "snow" in weather_desc:
        return "warm drink"
    else:
        return "normal food"

# Get workout type based on weather condition
def get_workout_type(weather_desc):
    weather_desc = weather_desc.lower()
    if "rain" in weather_desc or "storm" in weather_desc:
        return "indoor"
    elif "clear" in weather_desc or "sun" in weather_desc:
        return "outdoor"
    elif "snow" in weather_desc:
        return "strength"
    else:
        return "mixed"

# Get workout recommendation based on weather condition and exercise database
def get_workout_recommendation(weather_desc, exercise_df):
    # Print column names for debugging
    # print("\nAvailable exercise columns:", exercise_df.columns.tolist())

    workout_type = get_workout_type(weather_desc)

    # Check for required columns
    if "Exercise" not in exercise_df.columns or "PrimaryMuscle" not in exercise_df.columns:
        print("⚠️ Could not identify required columns in exercise data.")

        basic_recommendations = {
            "indoor": "Indoor workout: 3 sets of push-ups (15 reps), 3 sets of sit-ups (20 reps), 3 sets of lunges (10 each leg), 10 minutes of planks",
            "outdoor": "Outdoor activity: 30 minutes jogging, followed by bodyweight squats and lunges at a park",
            "strength": "Strength workout: 3 sets of push-ups (15 reps), pull-ups (8 reps), squats (20 reps), deadlifts with objects you have at home",
            "mixed": "Mixed workout: 15 min cardio, then 3 sets each: squats, push-ups, lunges, planks"
        }

        return basic_recommendations.get(workout_type, "General workout: 30 minutes cardio + basic strength exercises")

    # Create filters based on workout type
    if workout_type == "indoor":
        # For indoor, focus on exercises for upper body and core
        muscle_groups = ["Pectorals", "Deltoids", "Triceps", "Biceps", "Abdominals"]
        # Create a filter based on PrimaryMuscle column or individual muscle columns
        filtered_df = exercise_df[
            (exercise_df["PrimaryMuscle"].str.contains('|'.join(muscle_groups), case=False, na=False)) |
            (exercise_df[["Pectorals", "Deltoids", "Triceps", "Biceps", "Abdominals"]].isin(["Yes", "Some"]).any(axis=1))
        ]
        recommendation = "Indoor workout: Focus on upper body and core"
    elif workout_type == "outdoor":
        # For outdoor, focus on cardio and legs
        muscle_groups = ["Quad-riceps", "Ham-strings", "Gluteus", "Calves"]
        filtered_df = exercise_df[
            (exercise_df["PrimaryMuscle"].str.contains('|'.join(muscle_groups), case=False, na=False)) |
            (exercise_df[["Quad-riceps", "Ham-strings", "Gluteus", "Calves"]].isin(["Yes", "Some"]).any(axis=1))
        ]
        recommendation = "Outdoor activity: Focus on lower body exercises"
    elif workout_type == "strength":
        # For strength training, focus on major compound movements
        muscle_groups = ["Quad-riceps", "Ham-strings", "Gluteus", "Lats", "Pectorals"]
        filtered_df = exercise_df[
            (exercise_df["PrimaryMuscle"].str.contains('|'.join(muscle_groups), case=False, na=False)) |
            (exercise_df[["Quad-riceps", "Ham-strings", "Gluteus", "Lats", "Pectorals"]].isin(["Yes"]).any(axis=1))
        ]
        recommendation = "Strength workout: Focus on compound exercises"
    else:
        # Mixed approach - select a variety
        filtered_df = exercise_df
        recommendation = "Mixed workout: Full-body routine"

    # If filter returned empty dataframe, use the full dataset
    if len(filtered_df) == 0:
        filtered_df = exercise_df
        print("⚠️ No exercises matched the filter criteria, using all exercises")

    # Get 3-5 random exercises from filtered dataframe
    sample_size = min(5, len(filtered_df))
    sample_exercises = filtered_df.sample(n=sample_size)

    # Format exercises into routine
    exercises_list = []
    for _, exercise in sample_exercises.iterrows():
        # Build exercise string based on available columns
        exercise_text = f"{exercise['Exercise']}"

        # Add primary muscles if available
        if "PrimaryMuscle" in exercise and pd.notna(exercise['PrimaryMuscle']) and exercise['PrimaryMuscle']:
            exercise_text += f" (targets: {exercise['PrimaryMuscle']})"
        # Otherwise, try to identify primary muscles from individual columns
        else:
            muscles = []
            for col in exercise_df.columns:
                if col not in ["Exercise", "PrimaryMuscle", "Equipment"] and exercise.get(col) == "Yes":
                    muscles.append(col)
            if muscles:
                exercise_text += f" (targets: {', '.join(muscles)})"

        # Add equipment if available
        if "Equipment" in exercise and pd.notna(exercise['Equipment']):
            exercise_text += f" - Equipment: {exercise['Equipment']}"

        exercises_list.append(exercise_text)

    recommendation += "\n   Recommended exercises:"
    for i, ex in enumerate(exercises_list, 1):
        recommendation += f"\n   {i}. {ex}"

    # Add exercise prescription (sets and reps)
    recommendation += "\n\n   Suggested protocol:"
    if workout_type == "indoor":
        recommendation += "\n   • 3 sets of 10-12 reps for each exercise"
        recommendation += "\n   • 60 seconds rest between sets"
    elif workout_type == "outdoor":
        recommendation += "\n   • 4 sets of 15 reps for each exercise"
        recommendation += "\n   • Minimal rest between sets (30 seconds)"
    elif workout_type == "strength":
        recommendation += "\n   • 5 sets of 5 reps for each exercise"
        recommendation += "\n   • 2-3 minutes rest between sets"
    else:
        recommendation += "\n   • 3 sets of 12 reps for each exercise"
        recommendation += "\n   • 45-60 seconds rest between sets"

    return recommendation


# Fetch recipe from Spoonacular
recipe_cache = {}
def fetch_recipe(food_type):
    if food_type in recipe_cache:
        return random.choice(recipe_cache[food_type])
    url = f"https://api.spoonacular.com/recipes/complexSearch?query={food_type}&apiKey={SPOONACULAR_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        recipes = response.json()
        if recipes['results']:
            recipe_cache[food_type] = recipes['results']  # Store all recipes in cache
            return random.choice(recipes['results'])
    print(f"❌ Error fetching recipes: {response.status_code}")
    return None

# Fetch detailed recipe information
def fetch_recipe_details(recipe_id):
    url = f"https://api.spoonacular.com/recipes/{recipe_id}/information?apiKey={SPOONACULAR_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching recipe details: {response.status_code}")
    return None

def generate_text_with_cloudflare(prompt):
    headers = {
        "Authorization": f"Bearer {CLOUDFLARE_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "messages": [
            {"role": "system", "content": "You are an AI assistant that suggests meals and workouts based on weather."},
            {"role": "user", "content": prompt}
        ]
    }
    response = requests.post(CLOUDFLARE_AI_URL, json=payload, headers=headers)
    #print(response.json())
    if response.status_code == 200:
        return response.json()["result"]
    else:
        print(f"❌ Error with Cloudflare AI: {response.status_code}")
        return "No AI response available"

# Web scraping for alternative recipes
def scrape_recipes():
    # List of recipe URLs
    urls = {
        "soup": [
            "https://pinchofyum.com/sweet-potato-soup-with-roasted-cauliflower-crumbles",
            "https://pinchofyum.com/instant-pot-wild-rice-soup"
        ],
        "stew": [
            "https://pinchofyum.com/vegetarian-shepherds-pie",
            "https://pinchofyum.com/sunday-chili"
        ],
        "salad": [
            "https://pinchofyum.com/cauliflower-orange-gnocchi",
            "https://pinchofyum.com/chopped-thai-chicken-salad"
        ],
        "warm drink": [
            "https://pinchofyum.com/golden-milk",
            "https://pinchofyum.com/homemade-hot-chocolate"
        ],
        "normal food": [
            "https://pinchofyum.com/5-ingredient-honey-mustard-chicken-thighs",
            "https://pinchofyum.com/date-night-mushroom-pasta-with-goat-cheese"
        ]
    }

    # Headers to mimic a real browser request
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    # Dictionary to store extracted recipe data by food type
    recipes_by_type = {}

    for food_type, url_list in urls.items():
        recipes_by_type[food_type] = []
        for url in url_list:
            try:
                response = requests.get(url, headers=headers)
                time.sleep(1)  # Added 1 second delay here
                print(f"Adding a 1 second delay here...")
                soup = BeautifulSoup(response.text, "html.parser")

                # Extract title
                title_tag = soup.find("h1")
                title = title_tag.text.strip() if title_tag else "Title not found"

                # Extract ingredients
                ingredients_section = soup.find("div", class_="tasty-recipes-ingredients")
                ingredients = [li.text.strip() for li in ingredients_section.find_all("li")] if ingredients_section else ["Ingredients not found"]

                # Extract instructions
                instructions_section = soup.find("div", class_="tasty-recipes-instructions")
                instructions = [li.text.strip() for li in instructions_section.find_all("li")] if instructions_section else ["Instructions not found"]

                # Store in list
                recipes_by_type[food_type].append({
                    "title": title,
                    "ingredients": ingredients,
                    "instructions": instructions,
                    "url": url
                })
            except Exception as e:
                print(f"❌ Error scraping recipe from {url}: {str(e)}")

    return recipes_by_type

# Scrape Wikipedia for weight training exercises
def scrape_workout_exercises():
    try:
        print("🏋️ Scraping exercise data from Wikipedia...")
        # Wikipedia URL
        url = "https://en.wikipedia.org/wiki/List_of_weight_training_exercises"

        # Fetch page content
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")

        # Find all tables and locate the correct one that contains exercises and muscles
        tables = soup.find_all("table", {"class": "wikitable"})
        exercise_table = None

        for table in tables:
            # Look for tables with headers that match what we expect for exercise data
            headers = [th.text.strip() for th in table.find_all("th")]
            header_text = " ".join(headers).lower()

            # Check if this table contains muscle groups
            if "quadriceps" in header_text or "hamstrings" in header_text or "exercise" in header_text:
                exercise_table = table
                break

        if not exercise_table:
            print("❌ Could not find exercise table on Wikipedia page")
            raise Exception("Exercise table not found")

        # Extract headers - the first row contains headers
        headers = []
        header_row = exercise_table.find("tr")
        for th in header_row.find_all("th"):
            header_text = th.text.strip()
            # Clean up the header text - remove newlines and extra spaces
            header_text = ' '.join(header_text.split())
            headers.append(header_text)

        print(f"Found table with headers: {headers}")

        # Extract exercise names (first column) and muscle data (remaining columns)
        exercises = []
        muscle_data = []

        # Process each row after the header
        for row in exercise_table.find_all("tr")[1:]:  # Skip header row
            cells = row.find_all(["td", "th"])

            # Skip rows that don't have enough cells
            if len(cells) < 2:
                continue

            # First cell is typically the exercise name
            exercise_name = cells[0].text.strip()
            exercises.append(exercise_name)

            # Remaining cells contain muscle data
            row_data = {}
            for i, cell in enumerate(cells[1:], 1):
                if i < len(headers):  # Ensure we don't go out of bounds
                    muscle_group = headers[i]
                    value = cell.text.strip()
                    if value == "":
                        value = "No"  # Empty cells mean "No"
                    row_data[muscle_group] = value

            muscle_data.append(row_data)

        # Now we create a DataFrame with proper column names
        df_data = {"Exercise": exercises}

        # Add all muscle columns
        for muscle in headers[1:]:
            df_data[muscle] = []
            for row in muscle_data:
                df_data[muscle].append(row.get(muscle, "No"))

        # Create DataFrame
        df = pd.DataFrame(df_data)

        # Add a PrimaryMuscle column that combines the main muscles worked
        df["PrimaryMuscle"] = df.apply(
            lambda row: ", ".join([muscle for muscle in headers[1:]
                                  if muscle in row and row[muscle] == "Yes"]),
            axis=1
        )

        # If PrimaryMuscle is empty, include "Some" muscles
        df.loc[df["PrimaryMuscle"] == "", "PrimaryMuscle"] = df.apply(
            lambda row: ", ".join([muscle for muscle in headers[1:]
                                  if muscle in row and row[muscle] == "Some"]),
            axis=1
        )

        # Add Equipment column based on exercise name
        equipment_map = {
            "Squat": "Barbell, Squat rack",
            "Leg press": "Leg press machine",
            "Lunge": "Dumbbells or barbell (optional)",
            "Deadlift": "Barbell or dumbbells",
            "Leg extension": "Leg extension machine",
            "Leg curl": "Leg curl machine",
            "Standing calf raise": "Calf raise machine or platform",
            "Seated calf raise": "Seated calf raise machine",
            "Bench press": "Barbell or dumbbells, bench",
            "Chest fly": "Cable machine or dumbbells",
            "Push-up": "None",
            "Pull-down": "Lat pulldown machine",
            "Pull-up": "Pull-up bar",
            "Bent-over row": "Barbell or dumbbells",
            "Upright row": "Barbell or dumbbells",
            "Shoulder press": "Barbell or dumbbells",
            "Lateral raise": "Dumbbells",
            "Shoulder shrug": "Barbell or dumbbells",
            "Pushdown": "Cable machine",
            "Triceps extension": "Dumbbells or cable",
            "Biceps curl": "Dumbbells or barbell",
            "Crunch": "None",
            "Russian twist": "None",
            "Leg raise": "None",
            "Back extension": "Roman chair or hyperextension bench"
        }

        df["Equipment"] = df["Exercise"].map(equipment_map)
        df["Equipment"].fillna("None", inplace=True)

        print(f"✅ Successfully scraped {len(df)} exercises!")

        # Save to CSV for caching
        df.to_csv("weight_training_exercises.csv", index=False)

        return df

    except Exception as e:
        print(f"❌ Error scraping workout exercises: {str(e)}")

        # Try to load from CSV if scraping fails
        try:
            df = pd.read_csv("weight_training_exercises.csv")
            print("✅ Loaded exercise data from cached CSV file")
            return df
        except Exception as csv_error:
            print(f"❌ Could not load cached exercise data: {str(csv_error)}")

            # Create a simple default exercise DataFrame if all else fails
            print("⚠️ Creating basic exercise data for fallback")
            default_exercises = {
                "Exercise": [
                    "Push-ups", "Sit-ups", "Squats", "Lunges", "Plank",
                    "Pull-ups", "Bicep curls", "Tricep dips", "Jumping jacks",
                    "Mountain climbers"
                ],
                "PrimaryMuscle": [
                    "Pectorals, Triceps", "Abdominals", "Quadriceps, Gluteus", "Quadriceps, Gluteus", "Abdominals",
                    "Lats, Biceps", "Biceps", "Triceps", "Full body", "Abdominals"
                ],
                "Equipment": [
                    "None", "None", "None", "None", "None",
                    "Pull-up bar", "Dumbbells", "Bench/Chair", "None", "None"
                ]
            }
            return pd.DataFrame(default_exercises)

# Main function to generate recommendations
def generate_recommendations(weather_data, scraped_recipes, exercise_df):
    if not weather_data:
        return

    for i, forecast in enumerate(weather_data["list"][:24:8]):  # Limit to 3 days
        weather_desc = forecast["weather"][0]["description"]
        date = forecast["dt_txt"]
        temp = forecast["main"]["temp"]

        # Get food type and workout recommendation
        food_type = get_food_type(weather_desc)
        workout = get_workout_recommendation(weather_desc, exercise_df) if exercise_df is not None else "No exercise data available"

        # Fetch recipe from Spoonacular
        recipe = fetch_recipe(food_type)
        recipe_title = "No Recipe Found"
        recipe_url = "N/A"
        if recipe:
            recipe_details = fetch_recipe_details(recipe['id'])
            if recipe_details:
                recipe_title = recipe_details.get('title', 'Unknown Recipe')
                recipe_url = recipe_details.get('sourceUrl', 'No URL available')

        # Get alternative recipe from scraped data
        alt_recipe = None
        if food_type in scraped_recipes and scraped_recipes[food_type]:
            alt_recipe = random.choice(scraped_recipes[food_type])

        # Generate AI recommendation with Cloudflare
        prompt = f"The weather on {date} in {CITY} is {weather_desc} with a temperature of {temp}°C. Suggest a healthy meal and an exercise routine based on this weather."
        ai_response = generate_text_with_cloudflare(prompt)





        if isinstance(ai_response, dict) and 'response' in ai_response:
            ai_text = ai_response['response']  # Extract the actual suggestion text
        else:
            ai_text = str(ai_response)




        # Print recommendations
        print(f"\n{'='*80}")
        print(f"📅 Date: {date}")
        print(f"🌤 Weather: {weather_desc} | Temperature: {temp}°C")
        print(f"\n🍽 API RECOMMENDED RECIPE:")
        print(f"   {recipe_title} ({recipe_url})")

        print(f"\n💪 WORKOUT RECOMMENDATION:")
        print(f"   {workout}")

        #print(f"\n🤖 AI SUGGESTION: {ai_response}")

        print("\n🤖 **AI SUGGESTION:**")
        print(ai_text)

        # Print alternative recipe
        if alt_recipe:
            print(f"\n🍲 ALTERNATIVE RECIPE: {alt_recipe['title']}")
            print(f"   URL: {alt_recipe['url']}")

            print("\n   Ingredients:")
            for ing in alt_recipe['ingredients']:
                print(f"   - {ing}")

            print("\n   Instructions:")
            for step, instr in enumerate(alt_recipe['instructions'], 1):
                print(f"   {step}. {instr}")
        else:
            print("\n❌ No alternative recipe found for this weather condition")

        print(f"{'='*80}")

# Run the program
if __name__ == "__main__":
    if verify_cloudflare_token():
        print("🔍 Scraping alternative recipes...")
        scraped_recipes = scrape_recipes()
        print("✅ Recipe scraping complete!")

        # Scrape workout exercises from Wikipedia
        exercise_df = scrape_workout_exercises()

        CITY = get_user_city()
        print(f"🌤 Fetching weather data for {CITY}...")
        weather_data = fetch_weather_data(CITY)

        print("🧩 Generating personalized recommendations...")
        generate_recommendations(weather_data, scraped_recipes, exercise_df)

✅ Cloudflare API Token Verified!
🔍 Scraping alternative recipes...
Adding a 1 second delay here...
Adding a 1 second delay here...
Adding a 1 second delay here...
Adding a 1 second delay here...
Adding a 1 second delay here...
Adding a 1 second delay here...
Adding a 1 second delay here...
Adding a 1 second delay here...
Adding a 1 second delay here...
Adding a 1 second delay here...
✅ Recipe scraping complete!
🏋️ Scraping exercise data from Wikipedia...


<ipython-input-1-1ab9fd90911a>:421: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Equipment"].fillna("None", inplace=True)


Found table with headers: ['Exercise', 'Calves', 'Quad-riceps', 'Ham-strings', 'Gluteus', 'Hipsother', 'Lowerback', 'Lats', 'Trapezius', 'Abdominals', 'Pectorals', 'Deltoids', 'Triceps', 'Biceps', 'Forearms']
✅ Successfully scraped 25 exercises!
Enter your city: Mumbai
🌤 Fetching weather data for Mumbai...
🧩 Generating personalized recommendations...

📅 Date: 2025-03-04 00:00:00
🌤 Weather: clear sky | Temperature: 25.99°C

🍽 API RECOMMENDED RECIPE:
   Pork Schnitzel And Apple Salad (http://www.foodista.com/recipe/S7XYHPMB/pork-schnitzel-and-apple-salad)

💪 WORKOUT RECOMMENDATION:
   Outdoor activity: Focus on lower body exercises
   Recommended exercises:
   1. Back extension (targets: Ham-strings, Gluteus, Lowerback) - Equipment: Roman chair or hyperextension bench
   2. Seated calf raise (targets: Calves) - Equipment: Seated calf raise machine
   3. Leg press (targets: Quad-riceps, Gluteus) - Equipment: Leg press machine
   4. Lunge (targets: Quad-riceps, Ham-strings, Gluteus, Hipsot